# Setup

In [ ]:
!pip install torch
!pip install torch-geometric
!pip install captum # causes dependency issue with numpy as numpy requires a version <2; in colab simply hit restart runtime to use the older version without error
!pip install pandas
!pip install networkx
!pip install matplotlib
!pip install tqdm
!pip install numpy

In [ ]:
from itertools import pairwise, product
import networkx as nx
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric as pyg
from tqdm.auto import tqdm
import pandas as pd
import random
import matplotlib.pyplot as plt

In [ ]:
def set_seed(seed):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    return seed
set_seed(0)

In [ ]:
# Stores data with all values pertaining to their individual training
experiments = []

def add_experiment(param):
    experiments.append(param)

def get_dataframe():
    df = pd.DataFrame(experiments)
    return df

def clear_experiments():
    experiments.clear()

def save_dataframe(path):
    df = get_dataframe()
    df.to_csv(path)

In [ ]:
# This cell must change depending on runtime environment; it is currently configured for Colab.
from google.colab import drive
drive.mount("/content/drive")
folder_path = "/content/drive/MyDrive/FinDSExperiments/GINE_Size_Applicability/"

# Architecture

In [ ]:
class ShortestPathGNN(torch.nn.Module):
    def __init__(
        self,
        hidden_channels: int,
        num_layers: int
    ):
        super().__init__()
        in_channels = 1
        out_channels = 1
        self.encoder = torch.nn.Linear(in_channels, hidden_channels)

        self.layers = torch.nn.ModuleList()
        for _ in range(num_layers):
            layer = pyg.nn.GINEConv(
                nn=torch.nn.Sequential(
                    torch.nn.Linear(hidden_channels, hidden_channels),
                    torch.nn.ReLU(),
                    torch.nn.Linear(hidden_channels, hidden_channels),
                ),edge_dim=1)
            self.layers.append(layer)
        self.decoder = torch.nn.Linear(hidden_channels, out_channels)

    def forward(self, x: torch.Tensor, edge_index: torch.Tensor, edge_attr: torch.Tensor):
        x = self.encoder(x)
        for layer in self.layers:
            x = layer(x, edge_index, edge_attr)
        x = self.decoder(x)
        return x

# Generate Data

In [ ]:
from collections.abc import KeysView
K = 4
large_number = 10000

def convert_networkx_to_pyg_shortest_path(graph: nx.Graph,large_number:int):
    nx.set_edge_attributes(graph, values={e : 1.0 + 0.1*np.random.randn() for e in graph.edges()}, name='edge_attr')
    data = pyg.utils.convert.from_networkx(graph)
    data.x = torch.Tensor([0] + [large_number for _ in range(data.num_nodes-1)]).unsqueeze(1)
    length_dict = nx.shortest_path_length(graph, source=0, weight="edge_attr")
    data.y = torch.Tensor([length_dict.get(i, large_number) for i in range(data.num_nodes)])
    data.edge_attr = data.edge_attr.unsqueeze(1)
    return data

def get_connected_ER_graph(num_nodes: int, p: float):
    while True: # loop until we generate a connected graph
        graph = nx.erdos_renyi_graph(num_nodes, p)
        if nx.is_connected(graph):
            return graph

def create_pyg_dataset(num_nodes: int, num_samples: int, large_number:int):
    set_seed(0)
    return [
        convert_networkx_to_pyg_shortest_path(get_connected_ER_graph(num_nodes, p=0.1),large_number)
        for _ in range(num_samples)
    ]

###Custom experimental setup code:

In [ ]:
training_node_sizes = []
testing_node_sizes = []

value = 20.0;
while(value<1000):
    training_node_sizes.append(int(value))
    value*=1.2

value = 5.0;
while(value<2720):
    testing_node_sizes.append(int(value))
    value *= 1.3

In [ ]:
set_seed(0)

# Generate testing data
testing_data_by_sizes = {}
for testing_node_size in testing_node_sizes:
    num_test_samples = 100
    test_dataset = create_pyg_dataset(num_nodes=testing_node_size, num_samples=num_test_samples,large_number=large_number)
    testing_data_by_sizes[testing_node_size] = test_dataset

In [ ]:
set_seed(0)

# Generate training data
training_data_by_sizes = {}
for training_node_size in training_node_sizes:
    num_train_samples = 100
    train_dataset = create_pyg_dataset(num_nodes=training_node_size, num_samples=num_train_samples,large_number=large_number)
    training_data_by_sizes[training_node_size] = train_dataset

# Training/Testing

In [ ]:
# Training basic setup
device = "cuda" if torch.cuda.is_available() else "cpu"

def runTraining(hidden_channels_choice, seed, alpha, training_node_size):
    set_seed(seed)
    network = ShortestPathGNN(hidden_channels=hidden_channels_choice, num_layers=K)
    network.to(device)
    optimizer = torch.optim.Adam(network.parameters(), lr=0.0003)
    loss_function = torch.nn.MSELoss()
    num_epochs = 1000

    train_loader = pyg.loader.DataLoader(training_data_by_sizes[training_node_size], batch_size=64, shuffle=True)

    test_loader_temp = pyg.loader.DataLoader(testing_data_by_sizes[14], batch_size=64, shuffle=False) # for testing during training

    # Lists to store epoch loss and L1 norm; not saved in these experiments
    epoch_losses = []
    l1_norms = []

    def train_one_epoch(network, optimizer, loss_function, train_loader, alpha, device):
        network.train()
        epoch_loss = 0
        for batch in train_loader:
            network.zero_grad()
            batch = batch.to(device)
            pred = network(batch.x, batch.edge_index, batch.edge_attr)
            loss = loss_function(pred.flatten(), batch.y)

            # Add L1 regularization
            l1_norm = sum(p.abs().sum() for p in network.parameters())
            loss = loss + alpha * l1_norm

            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

        epoch_loss /= len(training_data_by_sizes[training_node_size])
        return epoch_loss, alpha * l1_norm.item(), l1_norm.item()

    def test_model(network, loss_function, test_loader, device):
        network.eval()
        test_loss = 0.
        with torch.no_grad():
            for batch in test_loader:
                batch = batch.to(device)
                pred = network(batch.x, batch.edge_index, batch.edge_attr)
                loss = loss_function(pred.flatten(), batch.y)
                test_loss += loss.item()
        test_loss /= len(test_loader.dataset)
        return test_loss

    pbar = tqdm(range(num_epochs))
    for epoch in pbar:
        train_loss, train_l1_norm, l1_norm = train_one_epoch(network, optimizer, loss_function, train_loader, alpha, device)
        test_loss_temp = test_model(network, loss_function, test_loader_temp, device)
        epoch_losses.append(train_loss)
        l1_norms.append(train_l1_norm)
        pbar.set_description(f"Large Number: {large_number}, Alpha: {alpha}, Hidden Channels: {hidden_channels_choice}, Epoch {epoch}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss_temp:.4f}, L1 Norm: {train_l1_norm:.4f}, Unadjusted L1: {l1_norm:.4f}")

    test_losses = {}

    for key,testing_data in testing_data_by_sizes.items():
        test_loader = pyg.loader.DataLoader(testing_data, batch_size=64, shuffle=False)
        test_loss = test_model(network, loss_function, test_loader, device)
        experiment = {"_type":0,"alpha":alpha,"Training Data Nodes":training_node_size,"Training Data Samples":len(training_data_by_sizes[training_node_size]),"Testing Data Nodes":key,"Testing Data Samples":len(testing_data),"Seed":seed,"K":K,"Large Number":large_number}
        experiment.update({"_":"|"})
        final_values = {"Hidden Channels": hidden_channels_choice,"Epoch": epoch, "Train_Loss": train_loss, "Test Loss": test_loss, "L1 Norm": train_l1_norm, "Unadjusted L1": l1_norm}
        experiment.update(final_values)
        add_experiment(experiment)
        test_losses[key] = test_loss
    return test_losses

###Custom experimental setup code:

In [ ]:
hidden_channels = 4
alpha = 1.5
test_losses_by_training_node_size = {}

for training_node_size in training_node_sizes:
    test_losses = runTraining(hidden_channels,0,alpha,training_node_size)
    test_losses_by_training_node_size[training_node_size] = test_losses

In [ ]:
test_losses_data = []
row_labels = []
column_labels = []
for key, val in test_losses_by_training_node_size.items():
    if not column_labels:
        column_labels = list(val.keys())
    test_losses_data.append(list(val.values()))
    row_labels.append(key)

test_losses_array = np.array(test_losses_data)

panda_array = pd.DataFrame(test_losses_array, index=row_labels, columns=column_labels)
panda_array.to_csv(folder_path + "test_losses.csv")